In [ ]:
import ipywidgets as widgets
from IPython.display import display
from queue import Queue
from threading import Thread
import pyaudio
import json
from vosk import Model, KaldiRecognizer
import time

In [ ]:
messages = Queue()
recordings = Queue()

In [ ]:
CHANNELS = 2
FRAME_RATE = 44100
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()
    stream = p.open(format=AUDIO_FORMAT,
                   channels=CHANNELS,
                   rate=FRAME_RATE,
                   input=True,
                   frames_per_buffer=chunk)
    frames = []
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []
    stream.stop_stream()
    stream.close()
    p.terminate()

In [ ]:
#model = Model(model_name="vosk-model-en-us-0.22")
#rec = KaldiRecognizer(model, FRAME_RATE)
#rec.SetWords(True)

def speech_recognition(output):
    '''
    while not messages.empty():
        frames = recordings.get()
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]
        output.append_stdout(text + "\n")
        time.sleep(1)
    '''
    return

In [ ]:
record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

output = widgets.Output()

In [ ]:
def start_recording(data):
    messages.put(True)
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()


def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")

In [ ]:
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)